Libraries imports and Foursquare details
--

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup #to scrape the web

import re #standard RegEx lib
print('Libraries imported.')

Libraries imported.


In [1]:
print('Foursquare API Information loaded!')

Foursquare API Information loaded!


Exploring the dataset obtained from https://simplemaps.com/data/it-cities
--


In [4]:
#load the csv file
df = pd.read_csv('it.csv')
print('Done!')

Done!


In [5]:
#let's see how the data set is structured
df.groupby(by='admin').describe()

lat                                             \
                      count       mean       std        min        25%   
admin                                                                    
Abruzzo                 4.0  42.458334  0.137773  42.350000  42.362500   
Basilicata              2.0  40.650000  0.023571  40.633333  40.641666   
Calabria                5.0  38.810000  0.460314  38.100000  38.666667   
Campania                5.0  40.923333  0.180893  40.683333  40.833333   
Emilia-Romagna          9.0  44.580123  0.308049  44.063333  44.417500   
Friuli-Venezia Giulia   4.0  45.904861  0.178358  45.648611  45.870903   
Lazio                   6.0  41.986111  0.392770  41.466667  41.700000   
Liguria                 4.0  44.175000  0.229936  43.883333  44.058333   
Lombardy               12.0  45.552778  0.325036  45.116667  45.279167   
Marche                  5.0  43.366667  0.410454  42.850000  43.150000   
Molise                  1.0  41.566667       NaN  41.566667  41.566667   
Piedmont                8.0  45.189583  0.481889  44.383333  44.900000   
Puglia                  8.0  40.986597  0.420123  40.383333  40.594028   
Sardegna               12.0  39.939931  0.642043  39.167222  39.421181   
Sicilia                10.0  37.596667  0.431678  36.916667  37.358334   
Trentino-Alto Adige     3.0  46.372222  0.264750  46.066667  46.291667   
Tuscany                10.0  43.618333  0.373790  42.766667  43.450000   
Umbria                  2.0  42.850000  0.400693  42.566667  42.708333   
Valle d’Aosta           1.0  45.733333       NaN  45.733333  45.733333   
Veneto                  7.0  45.533373  0.326470  45.066667  45.427639   

                                                         lng             \
                             50%        75%        max count       mean   
admin                                                                     
Abruzzo                42.416667  42.512500  42.650000   4.0  13.862500   
Basilicata             40.650000  40.658333  40.666667   2.0  16.200000   
Calabria               38.900000  39.083333  39.300000   5.0  16.343333   
Campania               40.900000  41.066667  41.133333   5.0  14.580000   
Emilia-Romagna         44.666667  44.800000  45.016667   9.0  11.252006   
Friuli-Venezia Giulia  45.954445  45.988403  46.061944   4.0  13.319375   
Lazio                  42.000000  42.325000  42.416667   6.0  12.569445   
Liguria                44.200000  44.316666  44.416667   4.0   8.833333   
Lombardy               45.566666  45.787500  46.166667  12.0   9.591667   
Marche                 43.300000  43.633333  43.900000   5.0  13.430000   
Molise                 41.566667  41.566667  41.566667   1.0  14.650000   
Piedmont               45.183333  45.491667  45.933330   8.0   8.206250   
Puglia                 41.175000  41.291666  41.450000   8.0  16.843299   
Sardegna               39.889444  40.419306  40.916667  12.0   9.010972   
Sicilia                37.533333  37.962500  38.183333  10.0  14.088333   
Trentino-Alto Adige    46.516667  46.525000  46.533333   3.0  11.300000   
Tuscany                43.741667  43.870833  44.016667  10.0  10.895000   
Umbria                 42.850000  42.991667  43.133333   2.0  12.491667   
Valle d’Aosta          45.733333  45.733333  45.733333   1.0   7.333333   
Veneto                 45.450000  45.608334  46.145000   7.0  11.858532   

                                                                             \
                            std        min        25%        50%        75%   
admin                                                                         
Abruzzo                0.404231  13.366667  13.616667  13.933333  14.179167   
Basilicata             0.565685  15.800000  16.000000  16.200000  16.400000   
Calabria               0.554602  15.650000  16.100000  16.250000  16.583333   
Campania               0.265204  14.250000  14.333333  14.750000  14.783333   
Emilia-Romagna         0.956493   9.666667

In [6]:
#clean up the df
df.drop(columns=['capital', 'population', 'population_proper'], inplace=True)
print('Done.')

Done.


Split the prominent cities among nord, center and south for better visualisation.
A way to do so it to take the difference between the northmost city and the southmost city latitudes, then divide by 3 and bin the cities respectively.

In [7]:
#northmost city's latitude
minLatitude = round(df['lat'].min(), 2)
#southmost city's latitude
maxLatitude = round(df['lat'].max(),2)

diff = maxLatitude - minLatitude

split1 = round((maxLatitude - diff/3),2)
split2 = round((maxLatitude - 2/3*diff),2)

print('The tree bins are from {} to {}, to {}, to {}'.format(maxLatitude, split1, split2, minLatitude))

The tree bins are from 46.53 to 43.33, to 40.12, to 36.92


In [8]:
#split the df into 3 df for north, middle and south
#north
df_north = df[df['lat'] > split1]
df_n = df_north.sort_values(by='city', ascending=True).reset_index(drop=True)
df_n.head()

,city,lat,lng,country,iso2,admin
0,Alessandria,44.900000,8.616667,Italy,IT,Piedmont
1,Ancona,43.633333,13.500000,Italy,IT,Marche
2,Aosta,45.733333,7.333333,Italy,IT,Valle d’Aosta
3,Arezzo,43.416667,11.883333,Italy,IT,Tuscany
4,Asti,44.900000,8.200000,Italy,IT,Piedmont


In [9]:
#middle
df_middle = df[df['lat'] > split2]
df_middle = df_middle[~df_middle.lat.isin(df_north.lat)].sort_values(by='city', ascending=True).reset_index(drop=True)
df_middle.head()

,city,lat,lng,country,iso2,admin
0,Andria,41.216667,16.283333,Italy,IT,Puglia
1,Ascoli Piceno,42.850000,13.566667,Italy,IT,Marche
2,Avellino,40.900000,14.783333,Italy,IT,Campania
3,Bari,41.133333,16.850000,Italy,IT,Puglia
4,Barletta,41.316667,16.283333,Italy,IT,Puglia


In [10]:
#south
df_south = df[df['lat'] < split2]
df_s = df_south.sort_values(by='city', ascending=True).reset_index(drop=True)
df_s.head()

,city,lat,lng,country,iso2,admin
0,Agrigento,37.316667,13.566667,Italy,IT,Sicilia
1,Cagliari,39.216667,9.116667,Italy,IT,Sardegna
2,Caltanissetta,37.483333,14.066667,Italy,IT,Sicilia
3,Carbonia,39.167222,8.522222,Italy,IT,Sardegna
4,Catania,37.500000,15.100000,Italy,IT,Sicilia


A **Folium** screen will help visualize the cities.

In [16]:
#Rome coordinates to center map
rome_lat = 41.8719
rome_lon = 12.5674

# create map of Italy using latitude and longitude values
map_italy = folium.Map(location=[rome_lat, rome_lon], zoom_start=5.5)

# add markers to map
for lat, lng, city, admin in zip(df['lat'], df['lng'], df['admin'], df['city']):
    
    label = '{}, {}'.format(admin, city)
    label = folium.Popup(label, parse_html=True)
    
    if lat > split1:
        n='green'
    elif split1 > lat > split2:
        n='blue'
    else:
        n='red'
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=True,
        fill_color=n,      
        fill_opacity=0.7,
        parse_html=False).add_to(map_italy)  
    
map_italy.save('fig1.html')
map_italy


Let's now use the **Fourquare API** for each prominent city in each Italian macroarea and get the venues.
--

Let's define the function to get the venues nearby the city centre of each city.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
     #   print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        data_stream = results['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in data_stream]) #u gotta believe

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    #venue_list is different than venues_list ... so it's just a 2x for loop
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

print('Function loaded!')

Function loaded!


In [18]:
#north
df_north_venues = getNearbyVenues(names=df_n['city'],
                                   latitudes=df_n['lat'],
                                   longitudes=df_n['lng']
                                  )

print('done!', df_north_venues.shape)

done! (4785, 7)


In [19]:
df_north_venues.sort_values(by='City', ascending=True).head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alessandria,44.9,8.616667,HOP - mangiare di birra,44.913718,8.610327,Pub
29,Alessandria,44.9,8.616667,Ristorazione Sociale di Alessandria,44.919877,8.622058,Food Court
30,Alessandria,44.9,8.616667,Il Moscardo,44.917340,8.613797,Italian Restaurant
31,Alessandria,44.9,8.616667,Il Mulino,44.914820,8.618470,Pizza Place
32,Alessandria,44.9,8.616667,Pepenero,44.914907,8.614228,Burger Joint


In [20]:
#middle
df_middle_venues = getNearbyVenues(names=df_middle['city'],
                                   latitudes=df_middle['lat'],
                                   longitudes=df_middle['lng']
                                  )

print('done!', df_middle_venues.shape)

done! (1683, 7)


In [21]:
df_middle_venues.sort_values(by='City', ascending=True).head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Andria,41.216667,16.283333,Piazza Catuma (nome tradizionale),41.226146,16.296932,Plaza
1,Andria,41.216667,16.283333,Soip Amaur,41.225981,16.297593,Pizza Place
2,Andria,41.216667,16.283333,Umami Ristorante,41.239707,16.315749,Italian Restaurant
3,Andria,41.216667,16.283333,Cristal Palace Hotel,41.230241,16.300724,Hotel
4,Andria,41.216667,16.283333,Cinemars Andria,41.246013,16.295297,Movie Theater


In [22]:
#south
df_south_venues = getNearbyVenues(names=df_south['city'],
                                   latitudes=df_south['lat'],
                                   longitudes=df_south['lng']
                                  )

print('done!', df_south_venues.shape)

done! (896, 7)


In [23]:
df_south_venues.sort_values(by='City', ascending=True).head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
895,Agrigento,37.316667,13.566667,Ristorante Buon Gusto 2,37.288570,13.528565,Seafood Restaurant
871,Agrigento,37.316667,13.566667,Valle dei Templi,37.289632,13.592014,History Museum
870,Agrigento,37.316667,13.566667,Caffè Concordia,37.311347,13.577336,Café
869,Agrigento,37.316667,13.566667,Tempio Della Concordia,37.289614,13.589089,Historic Site
868,Agrigento,37.316667,13.566667,Tempio Di Ercole,37.290152,13.586726,Historic Site


Scrape the Foursquare API Docs for a list of all the venues categories.
---

In [24]:
#parse the foursquare page for venues categories 

#make the request
url = 'https://developer.foursquare.com/docs/build-with-foursquare/categories/'
data_stream = requests.get(url).text

#scrape
soup = BeautifulSoup(data_stream)
#venues_category_list = soup.find('img')
entries = soup.findAll('h3')

blocks = []

for i in entries:
    blocks.append(str(i))
    
blocks[:5]

['<h3>Arts &amp; Entertainment</h3>',
 '<h3>Amphitheater</h3>',
 '<h3>Aquarium</h3>',
 '<h3>Arcade</h3>',
 '<h3>Art Gallery</h3>']

Let's clear the list.

In [25]:
#trim the strings
#blocks = [blocks[i].strip('<h3></') for i in range(len(blocks))]

venues_cat = []

for i,j in enumerate(blocks):
    
    a = j[4:]
    b = a[:-5]
    venues_cat.append(b)

venues_cat[:5]

['Arts &amp; Entertainment',
 'Amphitheater',
 'Aquarium',
 'Arcade',
 'Art Gallery']

Use the headers to determine the cultural-related attractions.

In [26]:
#
headers=[]
for i,j in enumerate(venues_cat):

    if '&amp;' in j:
        
        headers.append(venues_cat[i])
        
headers[:5]

['Arts &amp; Entertainment',
 'College &amp; University',
 'General College &amp; University',
 'Fish &amp; Chips Shop',
 'Mac &amp; Cheese Joint']

In [27]:
#select useful headers
trim_list = []
non_trim_list = []

for i,j in enumerate(venues_cat):
    
    if '&amp;' in j:
        
        if 'Arts' in j:
            trim_list.append(i)
        elif 'University' in j:
            trim_list.append(i)
        elif 'Landscaping' in j:
            trim_list.append(i)
        elif 'Municipalities' in j:
            trim_list.append(i)
        elif 'Athletics' in j:
            trim_list.append(i)
        elif 'Other Places' in j:
            trim_list.append(i)
        else:
            non_trim_list.append(i)            

print('indexes to keep ', trim_list)
print('indexes to remove ', non_trim_list)
for i in trim_list:
    print('headers ', venues_cat[i])
    
forbidden_words = ['&amp;', 'Restaurant', 'Shop', 'Store', 'Bar', 'Bakery', 'BBQ', 'Stadium', 'Court', 'Bowling',
                   'Burger', 'Buffet', 'Gym', 'Auto', 'Car', 'Deli', 'Golf', 'Hockey', 'Ski', 'Breakfast', 'Casino',
                  'Hospital', 'Noodle', 'Multiplex', 'Diner', 'Dog', 'Studio', 'Rock', 'Food', 'Creperie', 'Water Park',
                  'Zoo', 'Soccer', 'Beach', 'General Entertainment', 'Business',
                  'Café']

#unfortunately the 4Square API has a pretty poor venues categorization, so some words need to be extracted by hand

indexes to keep  [0, 65, 97, 492, 582, 599, 717, 751]
indexes to remove  [250, 389, 491, 713, 775, 796, 805, 806, 887, 911]
headers  Arts &amp; Entertainment
headers  College &amp; University
headers  General College &amp; University
headers  Athletics &amp; Sports
headers  States &amp; Municipalities
headers  Professional &amp; Other Places
headers  Arts &amp; Crafts Store
headers  Construction &amp; Landscaping


In [28]:
#extrapolate a subset
subs1 = venues_cat[0:250]
subs2 = venues_cat[491:712]
subs3 = venues_cat[716:775]
venues_cat_trimmed = subs1 + subs2 + subs3

print('original list len', len(venues_cat_trimmed))

original list len 530


In [29]:
#remove headers (and restaurants :); let's use regular expressions
#convert list to a series to better handle the index
venuesCatSer = pd.Series(venues_cat_trimmed)

for i,j in enumerate(venuesCatSer):
    
    for m in forbidden_words:
    
        if re.search(m,j) is not None:
            
            venuesCatSer.drop(i, inplace=True, errors='ignore')
            
print('trimmed list len', len(venuesCatSer))

venuesCatSer[:5]

trimmed list len 289


1    Amphitheater
2        Aquarium
3          Arcade
4     Art Gallery
7          Circus
dtype: object

Let's filter the cities data frames using the trimmed list of venues scraped from Foursquare.

In [30]:
#north Italy
#filtering join

df_north_venues.rename(columns={'Venue Category':'VenueCategory'}, inplace=True)

print('before cut ', df_north_venues.shape)

dfNorthTrim = df_north_venues[df_north_venues['VenueCategory'].isin(venuesCatSer)]

print('after cut ', dfNorthTrim.shape)

dfNorthTrim.reset_index(drop=True, inplace=True)
dfNorthTrim.head()

before cut  (4785, 7)
after cut  (827, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,VenueCategory
0,Alessandria,44.900000,8.616667,Circolo Borsalino,44.911314,8.631403,Pool
1,Alessandria,44.900000,8.616667,Piazzetta Della Lega Lombarda,44.913718,8.613837,Plaza
2,Alessandria,44.900000,8.616667,Santa Maria di Castello,44.917732,8.615409,History Museum
3,Alessandria,44.900000,8.616667,Cittadella di Alessandria,44.920859,8.606586,Historic Site
4,Ancona,43.633333,13.500000,Arco di Traiano,43.625243,13.506663,Monument / Landmark


In [31]:
#middle Italy
df_middle_venues.rename(columns={'Venue Category':'VenueCategory'}, inplace=True)

print('before cut ', df_middle_venues.shape)

dfMiddleTrim = df_middle_venues[df_middle_venues['VenueCategory'].isin(venuesCatSer)]

print('after cut ', dfMiddleTrim.shape)

dfMiddleTrim.reset_index(drop=True, inplace=True)
dfMiddleTrim.head()

before cut  (1683, 7)
after cut  (336, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,VenueCategory
0,Andria,41.216667,16.283333,Piazza Catuma (nome tradizionale),41.226146,16.296932,Plaza
1,Andria,41.216667,16.283333,Cinemars Andria,41.246013,16.295297,Movie Theater
2,Andria,41.216667,16.283333,Sala Roma,41.224222,16.296742,Movie Theater
3,Ascoli Piceno,42.850000,13.566667,Piazza del Popolo,42.854637,13.575476,Plaza
4,Ascoli Piceno,42.850000,13.566667,Piazza Arringo,42.853616,13.577572,Plaza


In [32]:
#south Italy
df_south_venues.rename(columns={'Venue Category':'VenueCategory'}, inplace=True)

print('before cut ', df_south_venues.shape)

dfSouthTrim = df_south_venues[df_south_venues['VenueCategory'].isin(venuesCatSer)]

print('after cut ', dfSouthTrim.shape)

dfSouthTrim.reset_index(drop=True, inplace=True)
dfSouthTrim.head()

before cut  (896, 7)
after cut  (158, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,VenueCategory
0,Palermo,38.116667,13.366667,Quattro Canti,38.115611,13.361531,Historic Site
1,Palermo,38.116667,13.366667,Giardini Garibaldi,38.117734,13.368744,Park
2,Palermo,38.116667,13.366667,Fontana Pretoria,38.115429,13.361879,Sculpture Garden
3,Palermo,38.116667,13.366667,Orto Botanico,38.113301,13.373003,Garden
4,Palermo,38.116667,13.366667,Teatro Massimo,38.120184,13.357208,Opera House


Let's now segment and cluster these cities by means of their cultural attractions.
-

First, some handling of the categorical variables is needed, starting from the **North**.
-

In [39]:
#using pandas 1hot encoding
north_cities_onehot = pd.get_dummies(dfNorthTrim[['VenueCategory']], prefix='', prefix_sep='')

# add cities column back to dataframe
north_cities_onehot['City'] = dfNorthTrim['City'] 

# move cities column to the first column
fixed_columns_nord = [north_cities_onehot.columns[-1]] + list(north_cities_onehot.columns[:-1])
north_cities_onehot = north_cities_onehot[fixed_columns_nord]

fixed_columns_nord[:5]

['Winery', 'Aquarium', 'Art Gallery', 'Art Museum', 'Auditorium']

In [40]:
#define the size
north_cities_onehot.shape

(827, 82)

In [41]:
#group
df_north_grouped = north_cities_onehot.groupby('City').mean().reset_index()
df_north_grouped.head()

,City,Winery,Aquarium,Art Gallery,Art Museum,Auditorium,Baseball Field,Bay,Bistro,Bookstore,Botanical Garden,Boutique,Bridge,Campground,Canal,Capitol Building,Castle,Cemetery,Church,City Hall,College Arts Building,College Library,Concert Hall,Convention Center,Distillery,Event Space,Factory,Farm,Field,Flea Market,Fountain,Garden,Harbor / Marina,Historic Site,History Museum,Hot Spring,Indie Movie Theater,Indie Theater,Island,Jazz Club,Lake,Lighthouse,Military Base,Monument / Landmark,Mountain,Mountain Hut,Movie Theater,Museum,Music Venue,Neighborhood,Opera House,Other Great Outdoors,Outdoor Sculpture,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Planetarium,Playground,Plaza,Pool,Pool Hall,Public Art,Racetrack,River,Rugby Pitch,Scenic Lookout,Science Museum,Sculpture Garden,Skating Rink,Sports Club,Stables,Surf Spot,Theater,Theme Park Ride / Attraction,Track,Trail,University,Village,Vineyard,Waterfront,Wedding Hall
0,Alessandria,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.250000,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Ancona,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.076923,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.307692,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.153846,0.0,0.0,0.0,0.0,0.230769,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aosta,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.428571,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.142857,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arezzo,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Asti,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.571429,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's define the kmeans parameters.

In [42]:
# set number of clusters
kclusters = 5

north_city_clustering = df_north_grouped.drop('City', 1)

# run k-means clustering
kmeans_north = KMeans(n_clusters=kclusters, random_state=0).fit(north_city_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_north.labels_[0:10]

array([0, 2, 0, 4, 0, 2, 2, 1, 2, 2])

Let's see which are the most common cultural attractions nearby!

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_north_venues_sorted = pd.DataFrame(columns=columns)
df_north_venues_sorted['City'] = df_north_grouped['City']

for ind in np.arange(df_north_grouped.shape[0]):
    df_north_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_north_grouped.iloc[ind, :], num_top_venues)
    
df_north_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alessandria,Plaza,History Museum,Historic Site,Pool,Farm,College Library,Concert Hall,Convention Center,Distillery,Event Space
1,Ancona,Monument / Landmark,Plaza,Park,Fountain,Historic Site,Theater,Bistro,Farm,Concert Hall,Convention Center
2,Aosta,Historic Site,Church,Mountain,Museum,Plaza,Wedding Hall,Field,Convention Center,Distillery,Event Space
3,Arezzo,Stables,Park,Flea Market,Concert Hall,Convention Center,Distillery,Event Space,Factory,Farm,Field
4,Asti,Plaza,Art Museum,History Museum,Theater,Wedding Hall,Field,Concert Hall,Convention Center,Distillery,Event Space


In [ ]:
df_north_venues_sorted.to_csv('df_north_most_common.csv') #save to csv

In [45]:
# add clustering labels and city coordinates
df_north_venues_sorted.insert(0, 'CityLat', df_n.lat)
df_north_venues_sorted.insert(1, 'CityLng', df_n.lng)
df_north_venues_sorted.insert(2, 'ClusterLabels', kmeans_north.labels_)
df_north_venues_sorted.set_index('City', inplace=True)
df_north_venues_sorted.head()

,CityLat,CityLng,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,,
Alessandria,44.900000,8.616667,0,Plaza,History Museum,Historic Site,Pool,Farm,College Library,Concert Hall,Convention Center,Distillery,Event Space
Ancona,43.633333,13.500000,2,Monument / Landmark,Plaza,Park,Fountain,Historic Site,Theater,Bistro,Farm,Concert Hall,Convention Center
Aosta,45.733333,7.333333,0,Historic Site,Church,Mountain,Museum,Plaza,Wedding Hall,Field,Convention Center,Distillery,Event Space
Arezzo,43.416667,11.883333,4,Stables,Park,Flea Market,Concert Hall,Convention Center,Distillery,Event Space,Factory,Farm,Field
Asti,44.900000,8.200000,0,Plaza,Art Museum,History Museum,Theater,Wedding Hall,Field,Concert Hall,Convention Center,Distillery,Event Space


In [46]:
df_north_venues_sorted.to_csv('north_final_table.csv')
df_north_venues_sorted.ClusterLabels.max()

4

Then the **Middle**.
-

In [47]:
#using pandas 1hot encoding
middle_cities_onehot = pd.get_dummies(dfMiddleTrim[['VenueCategory']], prefix='', prefix_sep='')

# add cities column back to dataframe
middle_cities_onehot['City'] = dfMiddleTrim['City'] 

# move neighborhood column to the first column
fixed_columns_mid = [middle_cities_onehot.columns[-1]] + list(middle_cities_onehot.columns[:-1])
middle_cities_onehot = middle_cities_onehot[fixed_columns_mid]

fixed_columns_mid[:5]

['Winery', 'Arcade', 'Art Gallery', 'Art Museum', 'Bistro']

In [48]:
#define the size
middle_cities_onehot.shape

(336, 44)

In [49]:
#group
df_middle_grouped = middle_cities_onehot.groupby('City').mean().reset_index()
df_middle_grouped.head()

,City,Winery,Arcade,Art Gallery,Art Museum,Bistro,Bookstore,Botanical Garden,Boutique,Bridge,Campground,Castle,Cemetery,Church,City Hall,Convention Center,Cultural Center,Fountain,Garden,Harbor / Marina,Historic Site,History Museum,Indie Movie Theater,Lighthouse,Monument / Landmark,Mountain,Movie Theater,Museum,Music Venue,Opera House,Other Great Outdoors,Palace,Park,Performing Arts Venue,Playground,Plaza,Pool,Scenic Lookout,Sculpture Garden,Temple,Theater,Trail,Waterfall,Waterfront
0,Andria,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,Ascoli Piceno,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0
2,Avellino,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750000,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0
3,Bari,0.0,0.0,0.0,0.0,0.1,0.200000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.0,0.100000,0.000000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0
4,Barletta,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0


In [50]:
# set number of clusters
kclusters = 5

middle_city_clustering = df_middle_grouped.drop('City', 1)

# run k-means clustering
kmeans_mid = KMeans(n_clusters=kclusters, random_state=0).fit(middle_city_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_mid.labels_[0:10]

array([3, 1, 0, 1, 4, 1, 1, 3, 1, 2])

Let's see which are the most common cultural attractions nearby!

In [51]:
# create columns according to number of top venues
columns = ['City']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_middle_venues_sorted = pd.DataFrame(columns=columns)
df_middle_venues_sorted['City'] = df_middle_grouped['City']

for ind in np.arange(df_middle_grouped.shape[0]):
    df_middle_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_middle_grouped.iloc[ind, :], num_top_venues)

df_middle_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andria,Movie Theater,Plaza,Waterfront,Castle,Garden,Fountain,Cultural Center,Convention Center,City Hall,Church
1,Ascoli Piceno,Plaza,Historic Site,Theater,Bookstore,Museum,Waterfront,Cemetery,Garden,Fountain,Cultural Center
2,Avellino,Plaza,Theater,Waterfront,Campground,Garden,Fountain,Cultural Center,Convention Center,City Hall,Church
3,Bari,Bookstore,Plaza,Theater,Lighthouse,Bistro,Boutique,Park,Movie Theater,Waterfront,Cemetery
4,Barletta,Castle,Theater,Waterfront,Harbor / Marina,Garden,Fountain,Cultural Center,Convention Center,City Hall,Church


In [52]:
# add clustering labels and city coordinates
df_middle_venues_sorted.insert(0, 'CityLat', df_middle.lat)
df_middle_venues_sorted.insert(1, 'CityLng', df_middle.lng)
df_middle_venues_sorted.insert(2, 'ClusterLabels', kmeans_mid.labels_)
df_middle_venues_sorted.set_index('City', inplace=True)
df_middle_venues_sorted.head()

,CityLat,CityLng,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,,
Andria,41.216667,16.283333,3,Movie Theater,Plaza,Waterfront,Castle,Garden,Fountain,Cultural Center,Convention Center,City Hall,Church
Ascoli Piceno,42.850000,13.566667,1,Plaza,Historic Site,Theater,Bookstore,Museum,Waterfront,Cemetery,Garden,Fountain,Cultural Center
Avellino,40.900000,14.783333,0,Plaza,Theater,Waterfront,Campground,Garden,Fountain,Cultural Center,Convention Center,City Hall,Church
Bari,41.133333,16.850000,1,Bookstore,Plaza,Theater,Lighthouse,Bistro,Boutique,Park,Movie Theater,Waterfront,Cemetery
Barletta,41.316667,16.283333,4,Castle,Theater,Waterfront,Harbor / Marina,Garden,Fountain,Cultural Center,Convention Center,City Hall,Church


And the south.
-

In [53]:
#using pandas 1hot encoding
south_cities_onehot = pd.get_dummies(dfSouthTrim[['VenueCategory']], prefix='', prefix_sep='')

# add cities column back to dataframe
south_cities_onehot['City'] = dfSouthTrim['City'] 

# move neighborhood column to the first column
fixed_columns_sud = [south_cities_onehot.columns[-1]] + list(south_cities_onehot.columns[:-1])
south_cities_onehot = south_cities_onehot[fixed_columns_sud]

fixed_columns_sud[:5]

['Winery', 'Amphitheater', 'Art Gallery', 'Art Museum', 'Bistro']

In [54]:
#define the size
south_cities_onehot.shape

(158, 33)

In [55]:
#group
df_south_grouped = south_cities_onehot.groupby('City').mean().reset_index()
df_south_grouped.head()

,City,Winery,Amphitheater,Art Gallery,Art Museum,Bistro,Bookstore,Boutique,Cafeteria,Castle,Church,Event Service,Event Space,Farm,Fountain,Garden,Historic Site,History Museum,Movie Theater,Museum,Music Venue,Neighborhood,Opera House,Park,Pedestrian Plaza,Performing Arts Venue,Plaza,Radio Station,River,Scenic Lookout,Sculpture Garden,Theater,Theme Park
0,Agrigento,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.500000,0.3,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.1,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,0.000000
1,Cagliari,0.0,0.000000,0.071429,0.0,0.0,0.000000,0.0,0.071429,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.071429,0.071429,0.0,0.000000,0.071429,0.0,0.0,0.000000,0.214286,0.0,0.0,0.357143,0.000000,0.0,0.000000,0.000000,0.000000,0.071429
2,Caltanissetta,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,Carbonia,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,Catania,0.0,0.041667,0.000000,0.0,0.0,0.083333,0.0,0.000000,0.041667,0.041667,0.0,0.041667,0.0,0.083333,0.041667,0.166667,0.0,0.041667,0.000000,0.0,0.0,0.041667,0.000000,0.0,0.0,0.208333,0.041667,0.0,0.041667,0.041667,0.041667,0.000000


In [56]:
# set number of clusters
kclusters = 5

south_city_clustering = df_south_grouped.drop('City', 1)

# run k-means clustering
kmeans_south = KMeans(n_clusters=kclusters, random_state=0).fit(south_city_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_south.labels_[0:10]

array([0, 0, 4, 0, 0, 4, 0, 1, 1, 1])

Let's see which are the most common cultural attractions nearby!

In [57]:
# create columns according to number of top venues
columns = ['City']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_south_venues_sorted = pd.DataFrame(columns=columns)
df_south_venues_sorted['City'] = df_south_grouped['City']

for ind in np.arange(df_south_grouped.shape[0]):
    df_south_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_south_grouped.iloc[ind, :], num_top_venues)

df_south_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agrigento,Historic Site,History Museum,Performing Arts Venue,Theater,Art Museum,Bistro,Bookstore,Art Gallery,Fountain,Cafeteria
1,Cagliari,Plaza,Park,Theme Park,Art Gallery,Cafeteria,Garden,Museum,Historic Site,Pedestrian Plaza,Performing Arts Venue
2,Caltanissetta,Park,Theme Park,Garden,Amphitheater,Art Gallery,Art Museum,Bistro,Bookstore,Boutique,Cafeteria
3,Carbonia,Historic Site,Plaza,Garden,Amphitheater,Art Gallery,Art Museum,Bistro,Bookstore,Boutique,Cafeteria
4,Catania,Plaza,Historic Site,Bookstore,Fountain,Church,Garden,Theater,Castle,Movie Theater,Opera House


In [58]:
# add clustering labels and city coordinates
df_south_venues_sorted.insert(0, 'CityLat', df_s.lat)
df_south_venues_sorted.insert(1, 'CityLng', df_s.lng)
df_south_venues_sorted.insert(2, 'ClusterLabels', kmeans_south.labels_)
df_south_venues_sorted.set_index('City', inplace=True)
df_south_venues_sorted.head()

,CityLat,CityLng,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,,
Agrigento,37.316667,13.566667,0,Historic Site,History Museum,Performing Arts Venue,Theater,Art Museum,Bistro,Bookstore,Art Gallery,Fountain,Cafeteria
Cagliari,39.216667,9.116667,0,Plaza,Park,Theme Park,Art Gallery,Cafeteria,Garden,Museum,Historic Site,Pedestrian Plaza,Performing Arts Venue
Caltanissetta,37.483333,14.066667,4,Park,Theme Park,Garden,Amphitheater,Art Gallery,Art Museum,Bistro,Bookstore,Boutique,Cafeteria
Carbonia,39.167222,8.522222,0,Historic Site,Plaza,Garden,Amphitheater,Art Gallery,Art Museum,Bistro,Bookstore,Boutique,Cafeteria
Catania,37.500000,15.100000,0,Plaza,Historic Site,Bookstore,Fountain,Church,Garden,Theater,Castle,Movie Theater,Opera House


Map of **North** clustered cities.
-

In [149]:
# create map
north_clusters = folium.Map(location=[rome_lat, rome_lon], zoom_start=5.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_north_venues_sorted['CityLat'], df_north_venues_sorted['CityLng'], df_north_venues_sorted.index, df_north_venues_sorted['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(north_clusters)
       
north_clusters.save('north_custers.html')
north_clusters

Time to explore the clusters!

In [182]:
#summary df
df_north_clusters = pd.DataFrame([])

#c0
df_north_c0 = df_north_venues_sorted.loc[df_north_venues_sorted['ClusterLabels'] == 0, df_north_venues_sorted.columns[list(range(3, df_north_venues_sorted.shape[1]))]]
df_north_clusters = df_north_c0.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue, Cluster0'})

#c1
df_north_c1 = df_north_venues_sorted.loc[df_north_venues_sorted['ClusterLabels'] == 1, df_north_venues_sorted.columns[list(range(3, df_north_venues_sorted.shape[1]))]]
df_north_c1 = df_north_c1.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_north_clusters['Cluster1'] = df_north_c1['CulturalVenue']

#c2
df_north_c2 = df_north_venues_sorted.loc[df_north_venues_sorted['ClusterLabels'] == 2, df_north_venues_sorted.columns[list(range(3, df_north_venues_sorted.shape[1]))]]
df_north_c2 = df_north_c2.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_north_clusters['Cluster2'] = df_north_c2['CulturalVenue']

#c3
df_north_c3 = df_north_venues_sorted.loc[df_north_venues_sorted['ClusterLabels'] == 3, df_north_venues_sorted.columns[list(range(3, df_north_venues_sorted.shape[1]))]]
df_north_c3 = df_north_c3.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_north_clusters['Cluster3'] = df_north_c3['CulturalVenue']

#c4
df_north_c4 = df_north_venues_sorted.loc[df_north_venues_sorted['ClusterLabels'] == 4, df_north_venues_sorted.columns[list(range(3, df_north_venues_sorted.shape[1]))]]
df_north_c4 = df_north_c4.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_north_clusters['Cluster4'] = df_north_c4['CulturalVenue']

df_north_clusters.to_csv('north_clusters.csv')

df_north_clusters

,"CulturalVenue, Cluster0",Cluster1,Cluster2,Cluster3,Cluster4
1st Most Common Venue,Plaza,Park,Plaza,Plaza,Stables
2nd Most Common Venue,Art Gallery,Park,Park,College Library,Park
3rd Most Common Venue,Theater,Bookstore,Historic Site,Concert Hall,Flea Market
4th Most Common Venue,Bookstore,Plaza,Theater,Convention Center,Concert Hall
5th Most Common Venue,Theater,Wedding Hall,Bistro,Distillery,Convention Center
6th Most Common Venue,Park,Historic Site,Bookstore,Event Space,Distillery
7th Most Common Venue,Bookstore,Distillery,Castle,Factory,Event Space
8th Most Common Venue,Convention Center,Wedding Hall,Concert Hall,Farm,Factory
9th Most Common Venue,Distillery,Factory,Convention Center,Field,Farm
10th Most Common Venue,Event Space,Farm,Distillery,Flea Market,Field


**Middle**.
-

In [150]:
# create map
middle_clusters = folium.Map(location=[rome_lat, rome_lon], zoom_start=5.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_middle_venues_sorted['CityLat'], df_middle_venues_sorted['CityLng'], df_middle_venues_sorted.index, df_middle_venues_sorted['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(middle_clusters)
    
middle_clusters.save('middle_custers.html')       
middle_clusters

Time to explore the clusters!

In [183]:
#summary df
df_middle_clusters = pd.DataFrame([])

#c0
df_middle_c0 = df_middle_venues_sorted.loc[df_middle_venues_sorted['ClusterLabels'] == 0, df_middle_venues_sorted.columns[list(range(3, df_middle_venues_sorted.shape[1]))]]
df_middle_clusters = df_middle_c0.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue, Cluster0'})

#c1
df_middle_c1 = df_middle_venues_sorted.loc[df_middle_venues_sorted['ClusterLabels'] == 1, df_middle_venues_sorted.columns[list(range(3, df_middle_venues_sorted.shape[1]))]]
df_middle_c1 = df_middle_c1.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_middle_clusters['Cluster1'] = df_middle_c1['CulturalVenue']

#c2
df_middle_c2 = df_middle_venues_sorted.loc[df_middle_venues_sorted['ClusterLabels'] == 2, df_middle_venues_sorted.columns[list(range(3, df_middle_venues_sorted.shape[1]))]]
df_middle_c2 = df_middle_c2.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_middle_clusters['Cluster2'] = df_middle_c2['CulturalVenue']

#c3
df_middle_c3 = df_middle_venues_sorted.loc[df_middle_venues_sorted['ClusterLabels'] == 3, df_middle_venues_sorted.columns[list(range(3, df_middle_venues_sorted.shape[1]))]]
df_middle_c3 = df_middle_c3.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_middle_clusters['Cluster3'] = df_middle_c3['CulturalVenue']

#c4
df_middle_c4 = df_middle_venues_sorted.loc[df_middle_venues_sorted['ClusterLabels'] == 4, df_middle_venues_sorted.columns[list(range(3, df_middle_venues_sorted.shape[1]))]]
df_middle_c4 = df_middle_c4.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_middle_clusters['Cluster4'] = df_middle_c4['CulturalVenue']

df_middle_clusters.to_csv('middle_clusters.csv')

df_middle_clusters

,"CulturalVenue, Cluster0",Cluster1,Cluster2,Cluster3,Cluster4
1st Most Common Venue,Plaza,Plaza,Park,Movie Theater,Castle
2nd Most Common Venue,Theater,Plaza,History Museum,Plaza,Theater
3rd Most Common Venue,Castle,Plaza,Harbor / Marina,Waterfront,Waterfront
4th Most Common Venue,Harbor / Marina,Bookstore,Garden,Harbor / Marina,Harbor / Marina
5th Most Common Venue,Garden,Waterfront,Garden,Garden,Garden
6th Most Common Venue,Fountain,Castle,Cultural Center,Fountain,Fountain
7th Most Common Venue,Cultural Center,Garden,Cultural Center,Cultural Center,Cultural Center
8th Most Common Venue,Convention Center,Fountain,Convention Center,Convention Center,Convention Center
9th Most Common Venue,City Hall,Cultural Center,Church,City Hall,City Hall
10th Most Common Venue,Church,Convention Center,Cemetery,Church,Church


**South**.
-

In [162]:
# create map
south_clusters = folium.Map(location=[rome_lat, rome_lon], zoom_start=5.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_south_venues_sorted['CityLat'], df_south_venues_sorted['CityLng'], df_south_venues_sorted.index, df_south_venues_sorted['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(south_clusters)

south_clusters.save('south_custers.html')  
south_clusters

In [184]:
#summary df
df_south_clusters = pd.DataFrame([])

#c0
df_south_c0 = df_south_venues_sorted.loc[df_south_venues_sorted['ClusterLabels'] == 0, df_south_venues_sorted.columns[list(range(3, df_south_venues_sorted.shape[1]))]]
df_south_clusters = df_south_c0.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue, Cluster0'})

#c1
df_south_c1 = df_south_venues_sorted.loc[df_south_venues_sorted['ClusterLabels'] == 1, df_south_venues_sorted.columns[list(range(3, df_south_venues_sorted.shape[1]))]]
df_south_c1 = df_south_c1.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_south_clusters['Cluster1'] = df_south_c1['CulturalVenue']

#c2
df_south_c2 = df_south_venues_sorted.loc[df_south_venues_sorted['ClusterLabels'] == 2, df_south_venues_sorted.columns[list(range(3, df_south_venues_sorted.shape[1]))]]
df_south_c2 = df_south_c2.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_south_clusters['Cluster2'] = df_south_c2['CulturalVenue']

#c3
df_south_c3 = df_south_venues_sorted.loc[df_south_venues_sorted['ClusterLabels'] == 3, df_south_venues_sorted.columns[list(range(3, df_south_venues_sorted.shape[1]))]]
df_south_c3 = df_south_c3.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_south_clusters['Cluster3'] = df_south_c3['CulturalVenue']

#c4
df_south_c4 = df_south_venues_sorted.loc[df_south_venues_sorted['ClusterLabels'] == 4, df_south_venues_sorted.columns[list(range(3, df_south_venues_sorted.shape[1]))]]
df_south_c4 = df_south_c4.describe().transpose().drop(columns=['count', 'unique', 'freq']).rename(columns={'top':'CulturalVenue'})

df_south_clusters['Cluster4'] = df_south_c4['CulturalVenue']

df_south_clusters.to_csv('south_clusters.csv')

df_south_clusters

,"CulturalVenue, Cluster0",Cluster1,Cluster2,Cluster3,Cluster4
1st Most Common Venue,Plaza,Plaza,Scenic Lookout,River,Park
2nd Most Common Venue,Plaza,Theme Park,Theme Park,Theme Park,Park
3rd Most Common Venue,Theme Park,Garden,Garden,Garden,Museum
4th Most Common Venue,Theater,Amphitheater,Amphitheater,Amphitheater,Amphitheater
5th Most Common Venue,Farm,Art Gallery,Art Gallery,Art Gallery,Art Gallery
6th Most Common Venue,Garden,Art Museum,Art Museum,Art Museum,Event Service
7th Most Common Venue,Bistro,Bistro,Bistro,Bistro,Church
8th Most Common Venue,Art Gallery,Bookstore,Bookstore,Bookstore,Castle
9th Most Common Venue,Fountain,Boutique,Boutique,Boutique,Boutique
10th Most Common Venue,Cafeteria,Cafeteria,Cafeteria,Cafeteria,Cafeteria


**End!**